<a href="https://colab.research.google.com/github/galdir/classificacao_com_bert/blob/main/geracao_de_vetores_de_texto_com_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Instalar pacotes


In [1]:
!pip install requests gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# Reiniciar depois da instalação do gensim para poder carregar

In [ ]:
import os
os.kill(os.getpid(), 9)

# Carregar pacotes

In [1]:
import requests
import zipfile
import os
import gensim
from gensim.models import KeyedVectors

# Configuração


Download de um modelo do NILC http://nilc.icmc.usp.br/embeddings

In [2]:
MODEL_URL = "http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip"
ZIP_FILENAME = "cbow_s50.zip"      # Nome do arquivo zip a ser salvo/procurado
MODEL_FILENAME = "cbow_s50.txt"  # Nome do arquivo de modelo dentro do zip e após extração
IS_BINARY_FORMAT = False         # Modelo é formato texto (.txt)

wv_model = None # Variavel para o modelo


# Baixa o ARQUIVO ZIP do word2vec


In [3]:

print(f"Baixando '{ZIP_FILENAME}' para a pasta atual...")

response = requests.get(MODEL_URL, stream=True, timeout=300)
response.raise_for_status() # Checa erros HTTP
with open(ZIP_FILENAME, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)
print("Download do zip concluído.")


Baixando 'cbow_s50.zip' para a pasta atual...
Download do zip concluído.


# Descompacta o ARQUIVO word2vec

In [4]:
print(f"Descompactando '{MODEL_FILENAME}' de '{ZIP_FILENAME}'...")

with zipfile.ZipFile(ZIP_FILENAME, 'r') as zip_ref:
    if MODEL_FILENAME not in zip_ref.namelist():
          raise FileNotFoundError(f"'{MODEL_FILENAME}' não encontrado dentro de '{ZIP_FILENAME}'")
    # Extrai o arquivo para a pasta atual (sem especificar 'path')
    zip_ref.extract(MODEL_FILENAME)
print(f"'{MODEL_FILENAME}' extraído.")



Descompactando 'cbow_s50.txt' de 'cbow_s50.zip'...
'cbow_s50.txt' extraído.


# Carrega o modelo Word2Vec

In [5]:
print(f"\nCarregando modelo '{MODEL_FILENAME}'...")
wv_model = KeyedVectors.load_word2vec_format(MODEL_FILENAME, binary=IS_BINARY_FORMAT)
print("Modelo carregado!")
print(f"Vocabulário: {len(wv_model.index_to_key):,} palavras, Dimensão: {wv_model.vector_size}")


Carregando modelo 'cbow_s50.txt'...
Modelo carregado!
Vocabulário: 929,606 palavras, Dimensão: 50


# Busca por palavras similares

In [6]:
palavra = 'tempo'
similares = wv_model.most_similar(palavra, topn=5)
print(f"Palavras similares a '{palavra}':")
for p in similares:
  print(p)


Palavras similares a 'tempo':
('tem-po', 0.798141598701477)
('período', 0.7876189351081848)
('lempo', 0.7823120951652527)
('ano', 0.7751826047897339)
('mês', 0.7722440958023071)


In [7]:
palavra = 'brasil'
similares = wv_model.most_similar(palavra, topn=5)
print(f"Palavras similares a '{palavra}':")
for p in similares:
  print(p)



Palavras similares a 'brasil':
('japão', 0.8968338966369629)
('haiti', 0.8842310309410095)
('trapichão', 0.8785302042961121)
('fazendão', 0.8718751668930054)
('canadã¡.', 0.8690112233161926)


# Similaridade entre palavras

In [8]:
p1, p2 = 'carro', 'bicicleta'
sim = wv_model.similarity(p1, p2)
print(f"\nSimilaridade entre '{p1}' e '{p2}': {sim:.4f}")


Similaridade entre 'carro' e 'bicicleta': 0.4308


In [29]:
p1, p2 = 'carro', 'moto'
sim = wv_model.similarity(p1, p2)
print(f"\nSimilaridade entre '{p1}' e '{p2}': {sim:.4f}")



Similaridade entre 'carro' e 'moto': 0.4999


In [44]:
p1, p2 = 'moto', 'bicicleta'
sim = wv_model.similarity(p1, p2)
print(f"\nSimilaridade entre '{p1}' e '{p2}': {sim:.4f}")


Similaridade entre 'moto' e 'bicicleta': 0.7902


# Analogias

In [67]:
palavra_pos1 = 'ator'
palavra_neg1 = 'homem'

palavra_pos2 = 'mulher'

resultado_analogia = wv_model.most_similar(positive=[palavra_pos1, palavra_pos2], negative=[palavra_neg1], topn=10)
print(f"\nAnalogia: '{palavra_pos1}' - '{palavra_neg1}' + '{palavra_pos2}' ≈ ?")
print(f"Resultado:")
for p in resultado_analogia:
  print(p)


Analogia: 'ator' - 'homem' + 'mulher' ≈ ?
Resultado:
('atriz', 0.8469098210334778)
('fotógrafa', 0.8101131319999695)
('novata', 0.8086127638816833)
('dançarina', 0.8073809146881104)
('escritora', 0.7975529432296753)
('bailarina', 0.7927482724189758)
('dubladora', 0.7895463109016418)
('ex-modelo', 0.7877095341682434)
('lutadora', 0.780687689781189)
('supermodelo', 0.7770102620124817)


In [69]:
palavra_pos1 = 'tio'
palavra_neg1 = 'homem'

palavra_pos2 = 'mulher'

resultado_analogia = wv_model.most_similar(positive=[palavra_pos1, palavra_pos2], negative=[palavra_neg1], topn=10)
print(f"\nAnalogia: '{palavra_pos1}' - '{palavra_neg1}' + '{palavra_pos2}' ≈ ?")
print(f"Resultado:")
for p in resultado_analogia:
  print(p)


Analogia: 'tio' - 'homem' + 'mulher' ≈ ?
Resultado:
('sobrinha', 0.8224742412567139)
('namorada', 0.8083332180976868)
('ex-namorada', 0.8058035969734192)
('esposa', 0.8007521033287048)
('ex-esposa', 0.7953395843505859)
('mãe', 0.7949250340461731)
('tia-avó', 0.7925654053688049)
('enteada', 0.7914786338806152)
('companheira', 0.7903515696525574)
('filha', 0.7898650169372559)


In [70]:
palavra_pos1 = 'rei'
palavra_neg1 = 'homem'

palavra_pos2 = 'mulher'

resultado_analogia = wv_model.most_similar(positive=[palavra_pos1, palavra_pos2], negative=[palavra_neg1], topn=10)
print(f"\nAnalogia: '{palavra_pos1}' - '{palavra_neg1}' + '{palavra_pos2}' ≈ ?")
print(f"Resultado:")
for p in resultado_analogia:
  print(p)



Analogia: 'rei' - 'homem' + 'mulher' ≈ ?
Resultado:
('esposa', 0.7769603133201599)
('governanta', 0.7553685903549194)
('sucessora', 0.7502477169036865)
('sobrinha', 0.7424736618995667)
('filha', 0.737616777420044)
('concubina', 0.7366868257522583)
('satrapia', 0.7352634072303772)
('conterrânea', 0.7217562198638916)
('madrasta', 0.7210681438446045)
('múmia', 0.718513548374176)


# Similaridade entre sentenças

In [46]:
import numpy as np
from numpy.linalg import norm

In [47]:
# Assume que 'wv_model' já foi carregado de um passo anterior

def calcular_vetor_medio_frase(frase, modelo):
    """Calcula o vetor médio para uma frase."""
    palavras = frase.lower().split() # Divide a frase em palavras minúsculas
    vetores_palavras = []

    for palavra in palavras:
        if palavra in modelo: # Verifica se a palavra existe no vocabulário do modelo
            vetores_palavras.append(modelo[palavra])

    if not vetores_palavras: # Se nenhuma palavra da frase está no modelo
        # Retorna um vetor de zeros com a dimensão correta
        return np.zeros(modelo.vector_size)

    # Calcula a média dos vetores das palavras encontradas
    vetor_medio = np.mean(vetores_palavras, axis=0)
    return vetor_medio

def calcular_similaridade_cosseno(vetor1, vetor2):
    """Calcula a similaridade do cosseno entre dois vetores numpy."""
    # Lida com caso de vetores nulos (sem palavras encontradas)
    norma1 = norm(vetor1)
    norma2 = norm(vetor2)
    if norma1 == 0 or norma2 == 0:
        return 0.0 # Similaridade é 0 se um dos vetores for nulo

    # Fórmula da Similaridade do Cosseno: dot(A, B) / (||A|| * ||B||)
    similaridade = np.dot(vetor1, vetor2) / (norma1 * norma2)
    return similaridade

In [49]:
frase_A = "como está o tempo hoje"
frase_B = "qual a previsão do tempo para hoje"

vetor_A = calcular_vetor_medio_frase(frase_A, wv_model)
vetor_B = calcular_vetor_medio_frase(frase_B, wv_model)

# Calcula e imprime as similaridades
sim_AB = calcular_similaridade_cosseno(vetor_A, vetor_B)
print(f"Similaridade entre:\n '{frase_A}'\n '{frase_B}'\n -> {sim_AB:.4f}")

Similaridade entre:
 'como está o tempo hoje'
 'qual a previsão do tempo para hoje'
 -> 0.4516


In [50]:
frase_A = "eu quero uma pizza de queijo"
frase_B = "qual a previsão do tempo para hoje"

vetor_A = calcular_vetor_medio_frase(frase_A, wv_model)
vetor_B = calcular_vetor_medio_frase(frase_B, wv_model)

# Calcula e imprime as similaridades
sim_AB = calcular_similaridade_cosseno(vetor_A, vetor_B)
print(f"Similaridade entre:\n '{frase_A}'\n '{frase_B}'\n -> {sim_AB:.4f}")

Similaridade entre:
 'eu quero uma pizza de queijo'
 'qual a previsão do tempo para hoje'
 -> 0.1575


In [51]:
frase_A = "eu quero uma pizza de queijo"
frase_B = "vamos comer numa pizzaria hoje?"

vetor_A = calcular_vetor_medio_frase(frase_A, wv_model)
vetor_B = calcular_vetor_medio_frase(frase_B, wv_model)

# Calcula e imprime as similaridades
sim_AB = calcular_similaridade_cosseno(vetor_A, vetor_B)
print(f"Similaridade entre:\n '{frase_A}'\n '{frase_B}'\n -> {sim_AB:.4f}")

Similaridade entre:
 'eu quero uma pizza de queijo'
 'vamos comer numa pizzaria hoje?'
 -> 0.7369


In [52]:
frase_A = "eu quero uma pizza de queijo"
frase_B = "vamos comer comida italiana hoje?"

vetor_A = calcular_vetor_medio_frase(frase_A, wv_model)
vetor_B = calcular_vetor_medio_frase(frase_B, wv_model)

# Calcula e imprime as similaridades
sim_AB = calcular_similaridade_cosseno(vetor_A, vetor_B)
print(f"Similaridade entre:\n '{frase_A}'\n '{frase_B}'\n -> {sim_AB:.4f}")

Similaridade entre:
 'eu quero uma pizza de queijo'
 'vamos comer comida italiana hoje?'
 -> 0.7001
